<a href="https://colab.research.google.com/github/hirom/python_scripts/blob/master/Selenium_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 動的 Web サイトのスクレイピング

Yahoo news のトピックス一覧から、記事のタイトルを抜き出すことができた次は、EDINET を例にとりましょう。

EDINET で有価証券報告書を見れる会社の沿革が書いてある HTML 文書を保存しようと思います。

1. まず下準備として、[EDINET](http://disclosure.edinet-fsa.go.jp/) のサイトにアクセスします。
2. `書類検索` をクリックします。
3. 画面左の `全文検索` をクリックします。
4. 文字列に `沿革` と入力します。
5. 書類情報を指定するのところで、`書類種別を指定する` をクリックします。`有価証券報告書` にチェックをします。
6. `書類の目次範囲を指定する` をクリックし、大分類の最初の項を選択し、小分類の最初の項を選択します。
7. 検索をクリックします。

新しいページが開いたら、そのページの URL をコピーします。
URL は次のようになるでしょう。

ここが、今回のスタート地点です。

In [0]:
url = 'https://disclosure.edinet-fsa.go.jp/E01EW/BLMainController.jsp?uji.verb=W1E63031Search&uji.bean=ee.bean.W1E63030.EEW1E63031Bean&PID=W1E63030&TID=W1E63031&SESSIONKEY=1505008790454&stype=0&dcdSelect=12001&hcdSelect=01001&ycdSelect=03001400&tsbSdt=&kbn=1&lgKbn=2&pkbn=0&skbn=1&dskb=&askb=&dflg=0&iflg=0&preId=1&chr=%E6%B2%BF%E9%9D%A9&hbn=true&spf5=2&otd=12001&hcd=01001&ycd=03001400&sec=&scc=&snm=&spf1=1&spf2=1&iec=&icc=&inm=&spf3=1&fdc=&fnm=&spf4=1&cal=1&era=H&yer=&mon=&psr=1&pid=4'

![](https://github.com/takahiromiura/github.io_files/blob/master/image/EDINET.png?raw=1)

このページでは、有価証券報告書が企業別に並んでいます。
それでは、最初の `KISCO株式会社` の有価証券報告書をクリックしてみましょう。

![](https://github.com/takahiromiura/github.io_files/blob/master/image/EDINET_window.png?raw=1)

小さなウインドウが新たに開きました。

このサイトは、クリックするとページの一部分が開いたり、新しいウインドウが開いたりしています。
これは、 `JavaScript` という言語によるものです。ユーザーがページをクリックするなどによって、ページの内容が変わるような Web サイトを **動的 Web サイト** と呼びます。一方、最初から全てのコンテンツがページに表示されているものは **静的 Web サイト** と呼びます。

## Selenium

動的 Web サイトをスクレイピングするには、`JavaScript` に対処しなければいけません。ここでは、 `Selenium` を使いましょう。Selenium は元は自動テスト用に作られたもので、ブラウザ上のコンテンツをクリックする、検索バーにキーを入力するなどを自動で行ってくれます。

まず、Google Chrome のブラウザを自動で動かすドライバーを[ここから](https://sites.google.com/a/chromium.org/chromedriver/downloads)ダウンロードします。ダウンロードしたら、参照しやすい場所に置いておきます。

また、`pip install selenium` で selenium の python 用モジュールをインストールしておいてください。

ブラウザを立ち上げます。アラートが出る場合は、`続行` を押してください。

In [0]:
from selenium import webdriver
browser = webdriver.Chrome()

次の作業に移る前に、少し待ちましょう。
すぐに次の作業に移ってしまうと、まだコンテンツが出ていないのに作業を行ってしまい、エラーが起こりやすくなります。`implicitly_wait` で指定した秒まで、要素が出るまで待ちます。

In [0]:
browser.implicitly_wait(10)

先程の URL にアクセスします。

In [0]:
browser.get(url)

サイトのタイトルが　`EDINET` であることを `assert` を使って確かめます。
違う場合はエラーが返ってきます。

In [0]:
assert 'EDINET' in browser.title

Selenium でも CSS セレクタが使えます。まずは、有価証券報告書のリンクをクリックします。
有価証券報告書、企業名は sytle_pb_after というクラスの要素の中の a 要素にあることがわかります。 `find_elements_by_css_selector` は指定した要素のリストを返します。

![](https://github.com/takahiromiura/github.io_files/blob/master/image/EDINET_report.png?raw=1)

In [0]:
links = browser.find_elements_by_css_selector('.table_border_1 > a')

リストは、A 社の有価証券報告書、A 社の企業名、B 社の有価証券報告書、B 社の企業名、... 、
という順番になっています。
最初のリンクをクリックする前に、クリックする企業名を保存しておきます。

In [0]:
company_name = links[1].text
company_name

'ＫＩＳＣＯ株式会社'

`click` を使うと、マウスのクリックをしたことになります。
KISCO 株式会社の有価証券報告書を指定してから、クリックをしてみます。

In [0]:
links[0].click()

新しいウインドウが出ましたね。新しいウインドウに移るには、`switch_to_window` を使います。
`browser.window_handles` はウインドウのリストで、古いウインドウから順にリストに入っています。
新しいウインドウは 1 番目の要素ですので、こちらに切り替えます。

In [0]:
new_window = browser.window_handles[1]
browser.switch_to_window(new_window)

新しいウインドウに切り替わったので、沿革を表示させるために、左側の適当な場所をクリックしたいです。

とりあえず `a` 要素をとってくれば良い気がしますが、ちょっと待ってください。
このサイトでは `frame` 要素が使われているようです。frame 要素は現在の HTML の規格（HTML5) には廃止されましたが、それ以前に作られたサイトでは使っているものもあるようですね。

![](https://github.com/takahiromiura/github.io_files/blob/master/image/EDINET_frame.png?raw=1)

html は各 frame に入っているので、いうなれば、Web ページが frame 毎に分割されているようなものです。ウインドウを移動したように、各 frame に移動しないと、内部の html を参照することができません。

実際に、`a` 要素をとってきても何も指定されません。

In [0]:
browser.find_elements_by_css_selector('a')

[]

`switch_to_frame` を使って frame 間を移動します。

このページでは、まず上部と下部で frame が分かれています。
クリックしたいのは、下部なので、下部の frame 名 `viewFrame` を指定して、移動します。

さらに、下部の frame は左上、左下、右側の 3 つに分かれています。
左下の frame 名 `menuFrame2` を指定して、左下に移動します。

In [0]:
browser.switch_to_frame("viewFrame")
browser.switch_to_frame("menuFrame2")

`a` 要素を指定して、0番目（表紙）以外をクリックします。

In [0]:
browser.find_elements_by_css_selector('a')[1].click()

無事に企業情報が出てきました。
今度は右側の frame に移りたいので、一旦元に戻ってから、右側に移動していきます。

In [0]:
browser.switch_to.default_content()
browser.switch_to_frame("viewFrame")
browser.switch_to_frame("mainFrame")

各情報は style_pb_after というクラスの要素に入っているので、セレクタで指定します。
沿革はその中の 2 番目です。

In [0]:
enkaku = browser.find_elements_by_css_selector('.style_pb_after')[2]

沿革かどうか一応確かめましょう。`h3` 要素を指定して、`text` を確認します。

In [0]:
assert '沿革' in enkaku.find_element_by_css_selector('h3').text

要素に入っている HTML をとるには、`get_attribute` を使い、`innerHTML` を取り出します。

In [0]:
enkaku_html = enkaku.get_attribute("innerHTML")

辞書に保存しておきましょう。

In [0]:
enkaku_dict = {}
enkaku_dict[company_name] = enkaku_html

これで、JavaScript のあるサイトから無事 HTML を取り出すことができました！
後は、BeautifulSoup などを使って Yahoo news でやったことと同じようにパースするだけです。

このように、`Selenium` を使えば、動的 Web サイトから情報を取り出すことも可能です。